# 6.4. Lazy Initialization

Lazy = preguiçosa

Até agora, pode parecer que escapamos impunes ao sermos desleixados na configuração de nossas redes. Especificamente, fizemos as seguintes coisas não intuitivas, que podem não parecer que deveriam funcionar:

- Definimos as arquiteturas de rede sem especificar a dimensionalidade de entrada.

- Adicionamos camadas sem especificar a dimensão de saída da camada anterior.

- Nós até mesmo “inicializamos” esses parâmetros antes de fornecer informações suficientes para determinar quantos parâmetros nossos modelos deveriam conter.

Você pode se surpreender que nosso código seja executado. Afinal, não há como o framework de aprendizado profundo dizer qual seria a dimensionalidade de entrada de uma rede. O truque aqui é que o framework adia a inicialização , esperando até a primeira vez que passamos dados pelo modelo, para inferir os tamanhos de cada camada na hora.

Mais tarde, ao trabalhar com redes neurais convolucionais, essa técnica se tornará ainda mais conveniente, pois a dimensionalidade de entrada (por exemplo, a resolução de uma imagem) afetará a dimensionalidade de cada camada subsequente. Portanto, a capacidade de definir parâmetros sem a necessidade de saber, no momento da escrita do código, o valor da dimensão pode simplificar muito a tarefa de especificar e, subsequentemente, modificar nossos modelos. Em seguida, nos aprofundamos na mecânica da inicialização.

In [1]:
import torch
from torch import nn
from d2l import torch as d2l

Para começar, vamos instanciar um MLP.

In [19]:
net = nn.Sequential(nn.LazyLinear(256),     # out=256; in não inicializado (por isso é preguiçosa)
                    nn.ReLU(), 
                    nn.LazyLinear(10))     # out=10; in não inicializado

__Neste ponto, a rede não pode saber as dimensões dos pesos da camada de entrada porque a dimensão de entrada permanece desconhecida.__
Consequentemente, o framework ainda não inicializou nenhum parâmetro. Confirmamos tentando acessar os parâmetros abaixo.

In [7]:
[(name, param) for name, param in net.named_parameters()] # param.shape não pode ser inicializado (UninitializedParameter) pois
                                                          # não se sabe a dimensão da entrada.

[('0.weight', <UninitializedParameter>),
 ('0.bias', <UninitializedParameter>),
 ('2.weight', <UninitializedParameter>),
 ('2.bias', <UninitializedParameter>)]

In [11]:
net[0].state_dict() 

OrderedDict([('weight', <UninitializedParameter>),
             ('bias', <UninitializedParameter>)])

In [12]:
net[0].weight

<UninitializedParameter>

Em seguida, vamos passar dados pela rede para que o framework finalmente inicialize os parâmetros.

In [13]:
X = torch.rand(2, 20)
net(X)

net[0].weight.shape

torch.Size([256, 20])

In [14]:
net[0].weight

Parameter containing:
tensor([[-0.1764, -0.0312,  0.1332,  ..., -0.2085,  0.1282, -0.1506],
        [-0.0541,  0.0656,  0.0547,  ...,  0.0935,  0.0461, -0.1254],
        [ 0.0390,  0.1964,  0.1284,  ...,  0.1928,  0.0656,  0.1535],
        ...,
        [-0.1563, -0.1202,  0.1623,  ..., -0.1768, -0.0481, -0.0682],
        [-0.1600,  0.0095, -0.0811,  ..., -0.1861, -0.2071, -0.1281],
        [ 0.1852, -0.1985,  0.1667,  ..., -0.0451,  0.0840, -0.1559]],
       requires_grad=True)

Assim que sabemos a dimensionalidade de entrada, 20, o framework pode identificar o formato da matriz de peso da primeira camada inserindo o valor 20. Tendo reconhecido o formato da primeira camada, o framework prossegue para a segunda camada, e assim por diante pelo gráfico computacional até que todos os formatos sejam conhecidos. Observe que, neste caso, apenas a primeira camada requer inicialização preguiçosa (lazy initialization), mas o framework inicializa sequencialmente. 

In [15]:
net.state_dict() 

OrderedDict([('0.weight',
              tensor([[-0.1764, -0.0312,  0.1332,  ..., -0.2085,  0.1282, -0.1506],
                      [-0.0541,  0.0656,  0.0547,  ...,  0.0935,  0.0461, -0.1254],
                      [ 0.0390,  0.1964,  0.1284,  ...,  0.1928,  0.0656,  0.1535],
                      ...,
                      [-0.1563, -0.1202,  0.1623,  ..., -0.1768, -0.0481, -0.0682],
                      [-0.1600,  0.0095, -0.0811,  ..., -0.1861, -0.2071, -0.1281],
                      [ 0.1852, -0.1985,  0.1667,  ..., -0.0451,  0.0840, -0.1559]])),
             ('0.bias',
              tensor([ 0.1495, -0.2089, -0.0306, -0.1353,  0.0119,  0.2185, -0.1546,  0.0699,
                       0.1010, -0.1609, -0.0339,  0.1757,  0.0913, -0.0989, -0.1393,  0.2023,
                      -0.0366,  0.0281,  0.2025,  0.1631,  0.1626, -0.0452, -0.1678, -0.1703,
                       0.0532, -0.0869, -0.1046,  0.0229, -0.1678, -0.0026,  0.0524, -0.2209,
                       0.1722,  0.0615, 

In [17]:
[(name, param.shape) for name, param in net.named_parameters()] 

[('0.weight', torch.Size([256, 20])),
 ('0.bias', torch.Size([256])),
 ('2.weight', torch.Size([10, 256])),
 ('2.bias', torch.Size([10]))]

O método a seguir passa entradas fictícias pela rede para uma execução a seco para inferir todas as formas de parâmetros e subsequentemente inicializa os parâmetros. Ele será usado mais tarde quando inicializações aleatórias padrão não forem desejadas.

In [18]:
@d2l.add_to_class(d2l.Module)  #@save
def apply_init(self, inputs, init=None):
    self.forward(*inputs)
    if init is not None:
        self.net.apply(init)

# 6.4.1. Resumo

A inicialização preguiçosa (sem inicializar entradas, apenas saídas) pode ser conveniente, permitindo que o framework infira formas de parâmetros automaticamente, facilitando a modificação de arquiteturas e eliminando uma fonte comum de erros. Podemos passar dados pelo modelo (fazer net(X)) para fazer o framework finalmente inicializar os parâmetros.

# 6.4.2. Exercícios

# Ex. 1 - O que acontece se você especificar as dimensões de entrada para a primeira camada, mas não para as camadas subsequentes? Você obtém inicialização imediata?

In [20]:
import torch
from torch import nn
from d2l import torch as d2l

net = nn.Sequential(nn.LazyLinear(out_features=256),
                    nn.ReLU(),
                    nn.LazyLinear(out_features=10))

X = torch.rand(2, 20)
net[0](X)                # inicializando a primeira camada

tensor([[-5.5999e-02,  1.1579e+00, -6.6829e-02, -4.8113e-01, -9.6888e-02,
         -4.7763e-01,  3.5814e-01, -2.8906e-01, -4.3587e-01,  8.3621e-01,
         -2.7230e-01, -9.9133e-03, -2.1856e-01,  2.1296e-02, -8.0526e-01,
         -1.2539e-02, -1.2116e-01, -2.5448e-01,  7.4097e-01, -5.4865e-01,
         -4.8688e-01, -5.0381e-01, -3.8352e-01, -3.5394e-01, -4.2473e-03,
         -1.2277e-02,  4.4227e-01, -9.9328e-01, -5.4534e-02,  4.7397e-01,
          2.1136e-01,  4.9253e-02,  5.4018e-02,  3.7261e-01,  7.6454e-03,
         -7.5679e-02,  1.9628e-01,  3.2247e-01, -6.7219e-01,  1.9240e-01,
          2.3901e-01, -1.3985e-01, -3.4483e-02,  5.7895e-02,  1.5708e-01,
          1.0999e-01, -3.9320e-01, -1.3569e-01,  9.8050e-01,  2.2505e-02,
         -7.1109e-01, -1.2067e-01, -3.0946e-01, -7.9308e-02,  2.3654e-01,
         -4.1967e-01, -3.2983e-01,  7.5846e-01, -5.8786e-01, -5.4799e-01,
         -2.4840e-01, -3.0474e-01, -5.6555e-02, -1.0236e-01, -2.7964e-01,
         -3.4616e-01, -4.0941e-01, -8.

In [21]:
net

Sequential(
  (0): Linear(in_features=20, out_features=256, bias=True)
  (1): ReLU()
  (2): LazyLinear(in_features=0, out_features=10, bias=True)
)

Portanto, apenas a primeira camada é inicializada. Logo, a inicialização é imediata, entretanto, não completa, como ocorre ao fazer net(X).

In [22]:
net(X)            # saída y

tensor([[ 0.1736,  0.0443,  0.2501,  0.0017, -0.1467,  0.0742,  0.0157, -0.3642,
          0.0682, -0.0447],
        [ 0.1509,  0.0662,  0.1918, -0.0151, -0.1127,  0.1692, -0.0116, -0.4011,
          0.0029, -0.1410]], grad_fn=<AddmmBackward0>)

In [23]:
net

Sequential(
  (0): Linear(in_features=20, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=10, bias=True)
)

In [24]:
net[2]


Linear(in_features=256, out_features=10, bias=True)

-----------------------------------------------------------------------

Modo 2:

In [26]:
import torch
from torch import nn
from d2l import torch as d2l

net = nn.Sequential(nn.Linear(in_features=784,out_features=256),              # obs: Linear, não LazyLinear.
                    nn.ReLU(),
                    nn.LazyLinear(out_features=10))                           # possui apenas dados de saída.
                                                                              # Linear possui dados de saída e entrada


In [28]:
net

Sequential(
  (0): Linear(in_features=784, out_features=256, bias=True)
  (1): ReLU()
  (2): LazyLinear(in_features=0, out_features=10, bias=True)
)

In [30]:
[(name, param) for name, param in net.named_parameters()] 

[('0.weight',
  Parameter containing:
  tensor([[-0.0326,  0.0092, -0.0270,  ..., -0.0194, -0.0322, -0.0268],
          [ 0.0170,  0.0155, -0.0228,  ...,  0.0043,  0.0277,  0.0321],
          [-0.0199, -0.0063,  0.0275,  ...,  0.0247, -0.0281, -0.0113],
          ...,
          [ 0.0307,  0.0003,  0.0088,  ..., -0.0246, -0.0210, -0.0014],
          [ 0.0298, -0.0309, -0.0086,  ...,  0.0062, -0.0278,  0.0123],
          [-0.0188,  0.0007,  0.0310,  ...,  0.0129, -0.0321,  0.0162]],
         requires_grad=True)),
 ('0.bias',
  Parameter containing:
  tensor([ 1.1608e-02, -2.9992e-02,  1.1081e-02,  1.9998e-02, -1.8117e-02,
          -5.7659e-03, -2.4703e-02,  3.4264e-02, -2.3190e-02,  6.8504e-03,
          -1.4736e-02,  3.3089e-02, -4.1488e-03,  1.5972e-02,  3.3234e-03,
          -1.9828e-02, -8.8795e-03,  3.3190e-02,  2.6050e-02,  3.3226e-02,
          -1.1092e-02, -8.6198e-03, -1.3684e-02, -2.4210e-02, -9.0527e-03,
           2.8036e-02, -3.5576e-02, -2.7194e-02, -1.0425e-02,  2.9703e-0

# Ex. 2 - O que acontece se você especificar dimensões incompatíveis?

In [36]:
import torch
from torch import nn
from d2l import torch as d2l

net = nn.Sequential(nn.Linear(784,256),
                    nn.ReLU(),
                    nn.Linear(64,10)           # Camada compatível seria:  nn.Linear(256,10) 
                    )

X = torch.rand(2, 784)
net(X)                # inicializando a primeira camada

RuntimeError: mat1 and mat2 shapes cannot be multiplied (2x256 and 64x10)

Com dimensões incompatíveis a rede não funciona.

# Outras respostas

https://pandalab.me/archives/lazy_initialization